In [3]:

###########################


import json
from collections import defaultdict
from decimal import Decimal
from IPython.display import Markdown, display
import random

# Load your JSON data
with open("input.json", "r") as f:
    data = json.load(f)

# Mapping IDs to labels
division_map = {item["i"]: item["l"] for item in data["drivers"]["DIVISION"]}
resource_class_map = {item["i"]: item["l"] for item in data["drivers"]["RESOURCE_CLASS"]}

# Store table data
table_data = defaultdict(lambda: defaultdict(lambda: {"$": 0.0, "HC": 0.0}))

# Function to generate random multiplier
def generate_multipliers():
    current_hc_multiplier = random.randint(1, 4)
    final_dollar_multiplier = random.randint(5, 8)
    return current_hc_multiplier, final_dollar_multiplier

# Get random multipliers for current HC and final $
current_hc_multiplier, final_dollar_multiplier = generate_multipliers()
print("current_hc_multiplier, final_dollar_multiplier")
print(current_hc_multiplier, final_dollar_multiplier)

# Populate data with multipliers applied
for entry in data["data"]:
    drv = entry["drvs"]
    division_id = drv["DIVISION"]
    resource_class_id = drv["RESOURCE_CLASS"]
    year = entry["p"]

    group_label = division_map.get(division_id, f"Division {division_id}")
    class_label = resource_class_map.get(resource_class_id, f"Class {resource_class_id}")

    current_key = f"{year} Current"
    final_key = f"{year} Final"

    # Apply multipliers
    table_data[group_label][class_label][f"{current_key} $"] = Decimal(entry.get("d01", 0))  # No multiplier for current $
    table_data[group_label][class_label][f"{current_key} HC"] = Decimal(entry.get("f01", 0)) * current_hc_multiplier
    table_data[group_label][class_label][f"{final_key} $"] = Decimal(entry.get("d12", 0)) * final_dollar_multiplier
    table_data[group_label][class_label][f"{final_key} HC"] = Decimal(entry.get("f12", 0)) * current_hc_multiplier

# Define headers
headers = [
    "Group", "2025 Current $", "2025 Current HC", "2025 Final $", "2025 Final HC",
    "2026 Current $", "2026 Current HC", "2026 Final $", "2026 Final HC",
    "2027 Final $", "2027 Final HC", "2028 Final $", "2028 Final HC"
]

# Markdown table construction
rows = []
rows.append("| " + " | ".join(headers) + " |")
rows.append("|" + "|".join(["-" * len(h) for h in headers]) + "|")

def fmt_currency(val):
    return f"${val:.0f}" if val else "$0"

def fmt_int(val):
    return f"{int(val)}" if val else "0"

# Populate markdown rows
for division, resources in table_data.items():
    row = [f"**{division}**"]
    for year in ["2025", "2026"]:
        row.extend([ 
            fmt_currency(resources.get("Employees", {}).get(f"{year} Current $", 0)),
            fmt_int(resources.get("Employees", {}).get(f"{year} Current HC", 0)),
            fmt_currency(resources.get("Employees", {}).get(f"{year} Final $", 0)),
            fmt_int(resources.get("Employees", {}).get(f"{year} Final HC", 0))
        ])
    for year in ["2027", "2028"]:
        row.extend([ 
            fmt_currency(resources.get("Employees", {}).get(f"{year} Final $", 0)),
            fmt_int(resources.get("Employees", {}).get(f"{year} Final HC", 0))
        ])
    rows.append("| " + " | ".join(row) + " |")

    for resource in ["Employees", "Contingent"]:
        if resource in resources:
            row = [resource]
            for year in ["2025", "2026"]:
                row.extend([ 
                    fmt_currency(resources[resource].get(f"{year} Current $", 0)),
                    fmt_int(resources[resource].get(f"{year} Current HC", 0)),
                    fmt_currency(resources[resource].get(f"{year} Final $", 0)),
                    fmt_int(resources[resource].get(f"{year} Final HC", 0))
                ])
            for year in ["2027", "2028"]:
                row.extend([ 
                    fmt_currency(resources[resource].get(f"{year} Final $", 0)),
                    fmt_int(resources[resource].get(f"{year} Final HC", 0))
                ])
            rows.append("| " + " | ".join(row) + " |")

# Build markdown: THIS IS WHAT IS RETURNED TO THE AGENT BY THE TOOL
markdown = "\n".join(rows)

# Save to markdown file
with open("output_table_weighted.md", "w") as f:
    f.write(markdown)

print("✅ Markdown table saved to output_table_weighted.md")
print(" Done")


current_hc_multiplier, final_dollar_multiplier
2 7
✅ Markdown table saved to output_table_weighted.md
 Done
